D'abord : se logguer à un compte gmail \
login : eleveenmi@gmail.com  (i=1,2,3,..) \
mwd : pwdeleveEnmi (i=1,2,3,..)

# I) Detar du jeu d'entraînement

II) Choix d'une augmentation de données

III) Entraînement d'un réseau de neurones

IV) Visualisation des prédictions sur le jeu de validation

V) Sauvegarde du réseau et partage vers le compte central

VI) Comparaison sur le jeu de test

In [4]:
# pour colab :
auth.authenticate_user()
# pour accéder aux fichiers google (google sheet)
creds, _ = default()  # or use service account
gc = gspread.authorize(creds)

# montage du drive :
drive.mount('/content/drive')

Mounted at /content/drive


# III) Dashboard

In [ ]:
groupes = list(np.arange(0,14))

# Assuming `gc = gspread.authorize(creds)` already done
show_annotation_dashboard(groupes, cf.group2sheets, gc)

# IV) Récupération des images annotées

In [ ]:
import os
import shutil
import tarfile
from collections import defaultdict

import gspread
import pandas as pd
from google.colab import drive
from gspread_dataframe import get_as_dataframe

# Mount Google Drive
# drive.mount('/content/drive')

# Configuration (import or define locally)
group2sheet_url = cf.group2sheets

group2direction_and_starthour = cf.group2direction_and_starthour

classes_to_sample = [1, 20, 21, 30, 40, 50, 60, 61, 70, 74, 75, 82]  # Extend this list as needed
base_drive_path = '/content/drive/MyDrive/stage_secondes_2025'

# Create target directories
os.makedirs('classification', exist_ok=True)
for cls in classes_to_sample:
    os.makedirs(f'classification/{cls}', exist_ok=True)

# Authorize gspread (you must have already set up your credentials)
# gc = gspread.authorize(creds)  # Make sure creds is defined properly

for group, sheet_url in group2sheet_url.items():
    print(group)
    direction, starthour = group2direction_and_starthour[group]
    tar_name = f"{group}_{direction}_{starthour}.tar.gz"
    tar_path = os.path.join(base_drive_path, tar_name)

    print(f"\nProcessing group {group}: extracting {tar_name}...")

    try:
        # Load and filter sheet
        sheet = gc.open_by_url(sheet_url).sheet1
        df = get_as_dataframe(sheet).dropna(subset=["Classe", "Image"])
        df = df[df['Classe'].astype(int).isin(classes_to_sample)]
    except Exception as e:
        print(f"Failed to load sheet for group {group}: {e}")
        continue

    if df.empty:
        print(f"No valid entries for group {group}.")
        continue

    # Extract tar
    try:
        with tarfile.open(tar_path, "r:gz") as tar:
            tar.extractall()
    except Exception as e:
        print(f"Error extracting {tar_path}: {e}")
        continue

    # Move images to classification folders
    for _, row in df.iterrows():
        image_name = row['Image']
        cls = int(row['Classe'])
        timestamp = image_name.split('_')[1].replace('.jpg', '')
        image_path = os.path.join(f"{group}_{direction}_{starthour}", timestamp, image_name)

        if os.path.exists(image_path):
            shutil.copy(image_path, f"classification/{cls}/")
        else:
            print(f"Missing image {image_path}")

    # Clean up
    shutil.rmtree(f"{group}_{direction}_{starthour}", ignore_errors=True)



In [41]:
folder_path = 'classification'
print(f"Total files in subfolders: {count_files_in_subfolders(folder_path)}")

Total files in subfolders: 422


In [ ]:
root_folder = '/content/classification'
plot_first_image_in_subfolder(root_folder)


# IV) Split train val test :

In [42]:
import os
import shutil
from collections import defaultdict
from math import ceil

# Create base folders
os.makedirs('learning/train', exist_ok=True)
os.makedirs('learning/val', exist_ok=True)
os.makedirs('test', exist_ok=True)

classification_dir = 'classification'
learning_dir = 'learning'
test_dir = 'test'

for class_name in os.listdir(classification_dir):
    class_path = os.path.join(classification_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    # Create output class folders
    os.makedirs(os.path.join(learning_dir, 'train', class_name), exist_ok=True)
    os.makedirs(os.path.join(learning_dir, 'val', class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

    # Gather all images and their date-based split keys
    image_files = [f for f in os.listdir(class_path) if f.endswith('.jpg')]
    images_by_modulo = {0: [], 1: [], 2: []}

    for img in image_files:
        try:
            tsd = int(img.split('_')[1][:8])
            mod = tsd % 3
            images_by_modulo[mod].append(img)
        except Exception as e:
            print(f"Skipping malformed image name: {img}")
            continue

    total = len(image_files)
    n_test = ceil(total * 0.2)
    n_val = ceil(total * 0.2)
    count_test = count_val = 0

    for mod in [0, 1, 2]:
        for img in images_by_modulo[mod]:
            src = os.path.join(class_path, img)

            if count_test < n_test:
                dst = os.path.join(test_dir, class_name, img)
                shutil.copy(src, dst)
                count_test += 1
            elif count_val < n_val:
                dst = os.path.join(learning_dir, 'val', class_name, img)
                shutil.copy(src, dst)
                count_val += 1
            else:
                dst = os.path.join(learning_dir, 'train', class_name, img)
                shutil.copy(src, dst)


In [43]:
import os
import pandas as pd
from collections import defaultdict

base_dirs = {
    "train": "learning/train",
    "val": "learning/val",
    "test": "test"
}

counts = defaultdict(dict)
for split, path in base_dirs.items():
    for class_name in os.listdir(path):
        class_path = os.path.join(path, class_name)
        if os.path.isdir(class_path):
            n_images = len([f for f in os.listdir(class_path) if f.endswith('.jpg')])
            counts[class_name][split] = n_images

df = pd.DataFrame(counts).T.fillna(0).astype(int)
df.loc['TOTAL'] = df.sum()
print(df)


       train  val  test
1         66   23    23
50        12    4     4
60         2    1     1
30        21    8     8
82         2    2     2
70         0    0     1
40        24    8     8
74         6    2     2
75        42   15    15
21        26   10    10
20        21    8     8
61        21    8     8
TOTAL    243   89    90


#V) tar it and automatic share

In [44]:
import os
import tarfile
from google.colab import drive
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.oauth2 import service_account

# Mount Google Drive
# drive.mount('/content/drive')

# Tar the folders
def make_tar(folder_name):
    with tarfile.open(f"{folder_name}.tar.gz", "w:gz") as tar:
        tar.add(folder_name, arcname=os.path.basename(folder_name))

make_tar("learning")
make_tar("test")

# Move to your target Drive folder
drive_folder = "/content/drive/MyDrive/stage_secondes_2025"
os.makedirs(drive_folder, exist_ok=True)
shutil.move("learning.tar.gz", os.path.join(drive_folder, "learning.tar.gz"))
shutil.move("test.tar.gz", os.path.join(drive_folder, "test.tar.gz"))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/stage_secondes_2025/test.tar.gz'

In [46]:
ls /content/drive/MyDrive/stage_secondes_2025

0_S_08H00.tar.gz   13_S_09H00.tar.gz  4_S_09H00.tar.gz  8_E_09H00.tar.gz
10_S_09H10.tar.gz  1_S_10H00.tar.gz   5_S_09H20.tar.gz  9_O_09H00.tar.gz
11_S_09H04.tar.gz  2_S_08H20.tar.gz   6_S_09H40.tar.gz  learning.tar.gz
12_S_09H02.tar.gz  3_S_08H40.tar.gz   7_N_09H00.tar.gz  test.tar.gz


In [47]:
# Set up Drive API auth (uses your Colab credentials)
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import mimetypes

drive_service = build('drive', 'v3')

# Get the file ID for learning.tar.gz
def get_file_id_by_name(file_name, parent_folder_name="stage_secondes_2025"):
    query = f"name = '{file_name}' and trashed = false"
    results = drive_service.files().list(q=query, fields="files(id, name, parents)").execute()
    files = results.get("files", [])
    for file in files:
        if parent_folder_name in file.get("parents", []):
            return file["id"]
    return files[0]["id"] if files else None

file_id = get_file_id_by_name("learning.tar.gz")

# Share with eleveenm1..13@gmail.com
for i in range(1, 2):
    user_email = f"eleveenm{i}@gmail.com"
    drive_service.permissions().create(
        fileId=file_id,
        body={
            "type": "user",
            "role": "reader",
            "emailAddress": user_email
        },
        fields="id"
    ).execute()
    print(f"Shared with {user_email}")


Shared with eleveenm1@gmail.com
